In [1]:
!pip install python-docx pypdf faiss-cpu transformers torch tqdm sentence-transformers fastapi uvicorn pyngrok

In [ ]:
import os
from typing import List, Dict, Union, Optional
from docx import Document
from pypdf import PdfReader
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModel
import torch
import pickle
from pathlib import Path
import json
from datetime import datetime
import uuid
import logging
import time
from tqdm import tqdm
import uvicorn
from pyngrok import ngrok
from pydantic import BaseModel
from fastapi import FastAPI
from functools import wraps
import nest_asyncio
from google.colab import userdata, drive
import re
from contextlib import asynccontextmanager
import shutil

nest_asyncio.apply()

def timer_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"{func.__name__} took {execution_time:.2f} seconds to execute")
        logging.info(f"{func.__name__} took {execution_time:.2f} seconds to execute")
        return result
    return wrapper

class Section:
    def __init__(self, text: str, heading: Optional[str] = None, level: int = 0):
        self.text = text
        self.heading = heading
        self.level = level
        self.content = {"text": text, "lists": []}

    def get(self, key, default=None):
        return getattr(self, key, default)

class DocumentManager:
    """Enhanced DocumentManager with improved section processing and response formatting"""

    @staticmethod
    def mount_drive() -> str:
        """Mount Google Drive and create necessary base directory"""
        try:
            # Don't force remount if already mounted
            if not os.path.exists('/content/drive'):
                drive.mount('/content/drive')

            base_path = '/content/drive/My Drive/lifesciences'
            os.makedirs(base_path, exist_ok=True)
            return base_path
        except Exception as e:
            logging.error(f"Failed to mount drive: {str(e)}")
            raise

    def __init__(self, model_name: str = "sentence-transformers/all-mpnet-base-v2", dimension: int = 768):
        self.base_path = self.mount_drive()
        self.setup_paths()
        self.setup_logging()
        self.setup_model(model_name)
        self.dimension = dimension
        self.index = None
        self.chunks = []
        self.initialize_index()
        self.section_pattern = re.compile(r'^(?:Section|Chapter|\d+\.|\d+\.\d+)\s+', re.IGNORECASE)
        print(f"DocumentManager initialized with base path: {self.base_path}")

    def setup_paths(self):
        """Setup and create necessary directories in Google Drive"""
        paths = {
            'models': 'models',
            'index': 'indexes',
            'extracted_data': 'extracted_data',
            'embeddings': 'embeddings',
            'training_docs': 'training_documents',
            'logs': 'logs'
        }

        for key, path in paths.items():
            full_path = os.path.join(self.base_path, path)
            setattr(self, f"{key}_path", full_path)
            os.makedirs(full_path, exist_ok=True)
            logging.info(f"Created directory: {full_path}")

    def setup_logging(self):
        """Setup logging configuration"""
        log_file = os.path.join(self.logs_path, 'processing.log')
        logging.basicConfig(
            filename=log_file,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        logging.info("Logging initialized")

    @timer_decorator
    def setup_model(self, model_name: str):
        """Initialize the transformer model and tokenizer"""
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModel.from_pretrained(model_name)
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(self.device)
            logging.info(f"Model loaded successfully on {self.device}")
        except Exception as e:
            logging.error(f"Error initializing model: {e}")
            raise

    def initialize_index(self):
        """Initialize or load existing FAISS index"""
        index_file = os.path.join(self.index_path, 'index.faiss')
        chunks_file = os.path.join(self.index_path, 'chunks.pkl')

        if os.path.exists(index_file) and os.path.exists(chunks_file):
            self.index = faiss.read_index(index_file)
            with open(chunks_file, 'rb') as f:
                self.chunks = pickle.load(f)
            logging.info(f"Loaded existing index with {len(self.chunks)} chunks")
        else:
            self.index = faiss.IndexFlatL2(self.dimension)
            self.chunks = []
            logging.info("Created new FAISS index")

    def extract_docx_sections(self, file_path: str) -> List[Dict]:
        """Extract sections from DOCX with null style handling"""
        try:
            doc = Document(file_path)
            sections = []
            current_section = None
            current_text = []

            logging.info(f"Starting DOCX extraction for {file_path}")
            logging.info(f"Total paragraphs: {len(doc.paragraphs)}")

            for i, paragraph in enumerate(doc.paragraphs):
                text = paragraph.text.strip()

                if not text:
                    continue

                # Safely get style information
                style_name = "default"
                try:
                    if paragraph.style and paragraph.style.name:
                        style_name = paragraph.style.name
                except AttributeError:
                    pass

                logging.info(f"Paragraph {i}: Style='{style_name}', Text='{text[:50]}...'")

                # Enhanced heading detection that doesn't rely on style
                is_heading = any([
                    style_name.startswith('Heading'),
                    len(text) <= 100 and text.isupper(),
                    bool(re.match(r'^[\d\.]+\s+[A-Z]', text)),
                    len(text.split()) <= 7 and text[0].isupper(),
                    bool(re.match(r'^[A-Z][A-Z\s]{4,}$', text))
                ])

                if is_heading:
                    # Save previous section if it exists
                    if current_text:
                        processed_content = self.process_lists('\n'.join(current_text))
                        if processed_content["text"] or processed_content["lists"]:
                            sections.append({
                                "heading": current_section,
                                "content": processed_content
                            })
                            logging.info(f"Added section with heading: {current_section}")

                    current_section = text
                    current_text = []
                else:
                    current_text.append(text)

            # Handle final section
            if current_text:
                processed_content = self.process_lists('\n'.join(current_text))
                if processed_content["text"] or processed_content["lists"]:
                    sections.append({
                        "heading": current_section,
                        "content": processed_content
                    })
                    logging.info(f"Added final section with heading: {current_section}")

            # Fallback for documents with no clear sections
            if not sections:
                all_text = []
                for para in doc.paragraphs:
                    text = para.text.strip()
                    if text:
                        all_text.append(text)

                if all_text:
                    processed_content = self.process_lists('\n'.join(all_text))
                    sections.append({
                        "heading": "Document Content",
                        "content": processed_content
                    })
                    logging.info("Created single section with all content")

            logging.info(f"Extracted {len(sections)} sections from DOCX")

            # Debug output of sections
            for i, section in enumerate(sections):
                logging.info(f"Section {i + 1}:")
                logging.info(f"  Heading: {section['heading']}")
                logging.info(f"  Content length: {len(section['content'].get('text', ''))}")
                logging.info(f"  Lists: {len(section['content'].get('lists', []))}")

            return sections

        except Exception as e:
            logging.error(f"Error in extract_docx_sections: {str(e)}")
            logging.exception("Full traceback:")

            # Attempt recovery by processing the entire document as one section
            try:
                all_text = []
                for para in doc.paragraphs:
                    text = para.text.strip()
                    if text:
                        all_text.append(text)

                if all_text:
                    processed_content = self.process_lists('\n'.join(all_text))
                    return [{
                        "heading": "Document Content",
                        "content": processed_content
                    }]
            except Exception as recovery_error:
                logging.error(f"Recovery attempt failed: {str(recovery_error)}")

            return []

    def is_heading(self, text: str) -> bool:
        """Enhanced heading detection with more patterns"""
        return any([
            bool(self.section_pattern.match(text)),
            text.isupper() and len(text.split()) <= 7,
            bool(re.match(r'^[\d\.]+\s+[A-Z]', text)),  # Numbered sections
            bool(re.match(r'^[A-Z][a-z]+\s+\d+', text)),  # "Section 1", "Chapter 2" etc.
            bool(re.match(r'^[A-Z][A-Z\s]{4,}$', text)),  # All caps words
            len(text.split()) <= 7 and text[0].isupper()  # Short capitalized phrases
        ])

    def extract_pdf_sections(self, file_path: str) -> List[Dict]:
        """Extract sections from PDF file and return in consistent format with DOCX extraction"""
        try:
            reader = PdfReader(file_path)
            sections = []
            current_section = None
            current_text = []

            for page in reader.pages:
                text = page.extract_text()
                if not text:
                    continue

                lines = text.split('\n')
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue

                    if self.is_heading(line):
                        # Save previous section if exists
                        if current_text:
                            sections.append({
                                "heading": current_section,
                                "content": self.process_lists('\n'.join(current_text))
                            })
                            current_text = []
                        current_section = line
                    else:
                        current_text.append(line)

            # Add the last section
            if current_text:
                sections.append({
                    "heading": current_section,
                    "content": self.process_lists('\n'.join(current_text))
                })

            # If no sections were found, create one with all content
            if not sections and current_text:
                sections.append({
                    "heading": None,
                    "content": self.process_lists('\n'.join(current_text))
                })

            logging.info(f"Extracted {len(sections)} sections from PDF: {file_path}")
            return sections

        except Exception as e:
            logging.error(f"Error extracting PDF sections from {file_path}: {str(e)}")
            return []


    def extract_docx_tables(self, file_path: str) -> List[Dict]:
        """Extract tables from DOCX file."""
        try:
            doc = Document(file_path)
            tables_data = []

            for table in doc.tables:
                table_list = []
                for row in table.rows:
                    row_list = []
                    for cell in row.cells:
                        row_list.append(cell.text)
                    table_list.append(row_list)
                tables_data.append(table_list)
            return tables_data
        except Exception as e:
            logging.error(f"Error extracting DOCX tables: {e}")
            return []

    def extract_pdf_tables(self, file_path: str) -> List[Dict]:
        """Extract tables from PDF using camelot."""
        try:
            import camelot  # Import camelot only when needed
            tables = camelot.read_pdf(file_path)
            tables_data = []
            for table in tables:
                df = table.df
                table_list = df.values.tolist()  # Convert DataFrame to list
                tables_data.append(table_list)
            return tables_data
        except Exception as e:
            logging.error(f"Error extracting PDF tables: {e}")
            return []

    def process_lists(self, text: str) -> Dict[str, Union[str, List[str]]]:
        """Process text to identify and structure lists"""
        lines = text.split('\n')
        processed_text = []
        current_list = []
        in_list = False

        list_markers = ['•', '-', '·', '*', '○', '▪', '✓']
        number_pattern = re.compile(r'^\d+[\.\)]')

        result = {
            "text": "",
            "lists": []
        }

        for line in lines:
            line = line.strip()
            if not line:
                continue

            is_list_item = (
                any(line.startswith(marker) for marker in list_markers) or
                number_pattern.match(line)
            )

            if is_list_item:
                if not in_list:
                    in_list = True
                    if processed_text:
                        result["text"] = '\n'.join(processed_text)
                current_list.append(line)
            else:
                if in_list:
                    in_list = False
                    if current_list:
                        result["lists"].append(current_list)
                        current_list = []
                processed_text.append(line)

        if in_list and current_list:
            result["lists"].append(current_list)
        if processed_text:
            result["text"] = '\n'.join(processed_text)

        return result

    @timer_decorator
    def get_embeddings(self, text: str) -> np.ndarray:
        """Generate embeddings for given text"""
        try:
            inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
            inputs = inputs.to(self.device)

            with torch.no_grad():
                outputs = self.model(**inputs)
                embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()

            return embedding
        except Exception as e:
            logging.error(f"Error generating embedding: {e}")
            return None

    def clear_directories(self):
        """Clears embeddings, indexes, and extracted data directories."""
        dirs_to_clear = [
            self.embeddings_path,
            self.index_path,
            self.extracted_data_path
        ]
        for dir_path in dirs_to_clear:
            try:
                if os.path.exists(dir_path):
                    shutil.rmtree(dir_path)  # Remove the directory and its contents
                    os.makedirs(dir_path)    # Recreate the directory
                    logging.info(f"Cleared directory: {dir_path}")
                else:
                    os.makedirs(dir_path)
                    logging.info(f"Directory created: {dir_path}")

            except Exception as e:
                logging.error(f"Error clearing directory {dir_path}: {e}")
                raise
    def process_document(self, file_path: str) -> Optional[str]:
        try:
            if not os.path.exists(file_path):
                logging.error(f"File not found: {file_path}")
                return None

            metadata = {  # Initialize metadata *before* processing sections
                "filename": os.path.basename(file_path),
                "file_path": file_path,
                "file_type": file_path.split('.')[-1].lower(),
                "processed": datetime.now().isoformat()
            }

            # Extract sections based on file type
            sections = []
            if file_path.endswith('.docx'):
                sections = self.extract_docx_sections(file_path)
            elif file_path.endswith('.pdf'):
                sections = self.extract_pdf_sections(file_path)
            else:
                logging.error(f"Unsupported file type: {file_path}")
                return None

            if not sections:
                logging.warning(f"No sections extracted from {file_path}")
                return None

            # Process each section
            for section_data in sections:
                # Handle both dictionary and Section object formats
                if isinstance(section_data, dict):
                    heading = section_data.get("heading")
                    content = section_data.get("content", {})
                else:
                    heading = section_data.heading
                    content = section_data.content

                print(f"content:{content}")
               # Ensure metadata is a dictionary (crucial!)
                if not isinstance(metadata, dict):
                    metadata = {}  # Initialize to an empty dict if it's not one

                logging.debug(f"Metadata before chunk creation: {metadata}")
                print(f"Metadata before chunk creation: {metadata}")
                # Process main text
                main_text = content.get("text", "")
                if main_text:
                    embedding = self.get_embeddings(main_text)
                    if embedding is not None:
                        self.index.add(embedding)
                        chunk = {
                            "text": main_text,
                            "metadata": metadata.copy(),  # Copy!
                            "heading": heading,
                            "type": "text"
                        }
                        self.chunks.append(chunk)
                        logging.debug(f"Added text chunk: {chunk}")  # Log the created chunk

                # Process lists (same logic as for main text)
                lists = content.get("lists", [])
                for lst in lists:
                    list_text = '\n'.join(lst) if isinstance(lst, list) else lst
                    embedding = self.get_embeddings(list_text)
                    if embedding is not None: # Simplified if check
                        self.index.add(embedding)
                        chunk = {
                            "text": list_text,
                            "metadata": metadata.copy(),  # Copy!
                            "heading": heading,
                            "type": "list"
                        }
                        self.chunks.append(chunk)
                        logging.debug(f"Added list chunk: {chunk}") # Log the created chunk



                # Extract and process tables
                tables = []
                if file_path.endswith('.docx'):
                    tables = self.extract_docx_tables(file_path)
                elif file_path.endswith('.pdf'):
                    tables = self.extract_pdf_tables(file_path)

                for table in tables:
                    table_text = "\n".join(["|".join(row) for row in table]) # Simple string representation
                    embedding = self.get_embeddings(table_text)
                    if embedding is not None:
                        self.index.add(embedding)
                        chunk = {
                            "text": table_text,
                            "metadata": metadata.copy(),
                            "heading": None,  # Tables might not have headings
                            "type": "table"
                        }
                        self.chunks.append(chunk)
                        logging.debug(f"Added table chunk: {chunk}")

            self.save_index_state()
            logging.info(f"Successfully processed document: {file_path}")
            return file_path

        except Exception as e:
            logging.exception(f"Error processing document {file_path}: {str(e)}")
            return None

    def save_index_state(self):
        """Save current index and chunks to disk"""
        index_file = os.path.join(self.index_path, 'index.faiss')
        chunks_file = os.path.join(self.index_path, 'chunks.pkl')

        faiss.write_index(self.index, index_file)
        with open(chunks_file, 'wb') as f:
            pickle.dump(self.chunks, f)
        logging.info(f"Saved index with {len(self.chunks)} chunks")

    @timer_decorator
    def query(self, query_text: str, top_k: int = 5) -> List[Dict[str, Union[str, float]]]:
        """Enhanced search with better response formatting"""
        try:
            query_embedding = self.get_embeddings(query_text)
            if query_embedding is not None:
                query_embedding = query_embedding.astype(np.float32)
                distances, indices = self.index.search(query_embedding, top_k)

                results = []
                for idx, dist in zip(indices[0], distances[0]):
                    if idx < len(self.chunks):
                        chunk = self.chunks[idx]
                        # CRITICAL: Check and log the chunk *before* accessing metadata
                        logging.debug(f"Chunk before metadata access: {chunk}")
                        if "metadata" not in chunk:  # Check if metadata exists!
                            logging.error(f"Chunk is missing metadata: {chunk}")
                            chunk["metadata"] = []
                            # Handle the error gracefully (e.g., skip this chunk)
                            #continue  # Skip to the next chunk

                        result = { # Now it's safe to access metadata
                            "content": chunk["text"],
                            "distance": float(dist),
                            "metadata": chunk["metadata"],
                            "type": chunk.get("type", "text"),
                            "heading": chunk.get("heading", "")
                        }
                        results.append(result)


                # Format response based on content type
                formatted_response = self.format_response(results)
                return [{"generated_text": formatted_response}]

            return [{"generated_text": "No results found"}]
        except Exception as e:
            logging.error(f"Error performing query: {e}")
            return [{"generated_text": "Error processing query"}]

    def format_response(self, results: List[Dict]) -> str:
        """Format search results based on content type"""
        formatted_sections = []

        for result in results:
            section = []
            if result["heading"]:
                section.append(f"## {result['heading']}")

            if result["type"] == "list":
                # Format list items
                items = result["content"].split('\n')
                section.extend(items)

            elif result["type"] == "table":  # Handle tables
                table_str = result["content"]
                section.append(table_str)  # Directly append table string

            else:
                # Format regular text
                section.append(result["content"])

            section.append("\n")  # Add spacing between sections
            formatted_sections.append('\n'.join(section))

        return '\n'.join(formatted_sections)

    def process_training_documents(self):
        """Process all documents in the training directory"""
        self.clear_directories()
        processed_files = []
        for filename in os.listdir(self.training_docs_path):
            if filename.endswith(('.docx', '.pdf')):
                file_path = os.path.join(self.training_docs_path, filename)
                if self.process_document(file_path):
                    processed_files.append(file_path)

        logging.info(f"Processed {len(processed_files)} training documents")
        return processed_files

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup (Only ngrok setup here)
    try:
        ngrok_token = userdata.get('ngrok_auth_token')
        if ngrok_token:
            ngrok.set_auth_token(ngrok_token)
            tunnel = ngrok.connect(8000)  # Establish the tunnel
            public_url = tunnel.public_url #Simplified
            logging.info(f"API available at: {public_url}")
            print(f"API available at: {public_url}") #Print to console too
            app.state.ngrok_url = public_url #Store it in app.state
        else:
            logging.warning("ngrok_auth_token not found in Colab userdata")

    except Exception as e:
        logging.error(f"Error in ngrok setup (lifespan): {e}")
        logging.exception("Full traceback:")  # Essential for debugging
        raise  # Re-raise to prevent app start if ngrok fails

    yield  # Important: Yield control to the app

    # Shutdown (Only ngrok disconnect here)
    try:
        if hasattr(app.state, 'ngrok_url'):  # Check if ngrok was initialized
            ngrok.disconnect(app.state.ngrok_url)
    except Exception as e:
        logging.error(f"Error in ngrok shutdown: {e}")
        logging.exception("Full traceback:")
# FastAPI Setup
app = FastAPI(lifespan=lifespan)

class QueryInput(BaseModel):
    inputs: str

document_manager = None

@app.post("/query")
async def query_endpoint(query: QueryInput):
    """Endpoint for querying the document database with improved response formatting"""
    global document_manager
    try:
        logging.info(f"Received query: {query.inputs}")
        response = document_manager.query(query.inputs, 10)
        return response
    except Exception as e:
        logging.error(f"Error in query endpoint: {e}")
        return [{"generated_text": "Error processing query"}]

@app.get("/train")
async def train_endpoint():
    """Endpoint for training/updating the document database"""
    global document_manager
    try:
        logging.info("Received training request")
        print("Initializing document manager...")
        document_manager = DocumentManager()
        processed_files = document_manager.process_training_documents()
        return [{"generated_text": f"Successfully processed {len(processed_files)} documents"}]
        document_manager.load_index_state()
    except Exception as e:
        logging.error(f"Error in train endpoint: {e}")
        return [{"generated_text": "Error processing training request"}]

def main():
    """Main function to initialize and start the application"""
    global document_manager
    try:
        # Initialize document manager
        print("Initializing document manager...")
        document_manager = DocumentManager()

        # Process training documents
        print("Processing training documents...")
        processed_files = document_manager.process_training_documents()

        if document_manager.index.ntotal > 0:
            print(f"Successfully processed {len(processed_files)} files")
            print("Starting server on http://0.0.0.0:8000")
            uvicorn.run(app, host="0.0.0.0", port=8000)
        else:
            error_msg = "No documents were processed. Server startup aborted."
            logging.error(error_msg)
            raise RuntimeError(error_msg)

    except Exception as e:
        error_msg = f"Error in main: {str(e)}"
        logging.error(error_msg)
        raise

if __name__ == "__main__":
    main()

Initializing document manager...
setup_model took 18.25 seconds to execute
DocumentManager initialized with base path: /content/drive/My Drive/lifesciences
Processing training documents...
content:{'text': 'Table of Contents\n1 General Requirements\n2 Automation\n3 Process Control\n4 Process Safety\n5 Machine Safety\n6 Vessels', 'lists': [['1.1 Utilites', '1.2 Facility Requirements', '1.3 Safety Requirements', '1.4 Environmental Requirements', '1.5 Calibration Requirements', '1.6 Documentation', '1.7 Training Requirements', '1.8 Vendor and Warranty Specifications'], ['2.1 Platform and Networking Requirements', '2.2 Hardware Requirements', '2.3 Software Requirements', '2.4 System Performance Requirements', '2.5 Historical Data Trending', '2.6 Alarm and Events', '2.7 Operational Requirements', '2.8 User Roles and Access Requirements', '2.9 Password Rules', '2.10 Time Synchronization', '2.11 Security Requirements', '2.12 Anti Virus and Patching', '2.13 Electronic Signatures (21 CFR 11)', 

ERROR:root:Error extracting PDF tables: No module named 'camelot'


content:{'text': '1General Requirements', 'lists': []}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T10:38:42.804897'}
get_embeddings took 0.11 seconds to execute
content:{'text': 'pharmaceutical manufacturing, includingGMP and FDA regulations.', 'lists': []}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T10:38:42.804897'}


ERROR:root:Error extracting PDF tables: No module named 'camelot'
ERROR:root:Error extracting PDF tables: No module named 'camelot'


get_embeddings took 0.15 seconds to execute
content:{'text': 'reliability during operational hours.', 'lists': []}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T10:38:42.804897'}
get_embeddings took 0.14 seconds to execute
content:{'text': 'time between batches by 20% compared to existing equipment.', 'lists': []}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T10:38:42.804897'}


...

get_embeddings took 0.20 seconds to execute
content:{'text': '', 'lists': [['6.10Cleaning and Maintenance', '6.11Operational Requirements', '6.12Regulatoryand Compliance Standards', '6.13Documentation and Record-Keeping']]}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T11:12:53.362706'}


ERROR:root:Error extracting PDF tables: No module named 'camelot'


get_embeddings took 0.28 seconds to execute
content:{'text': '', 'lists': [['6.14Training Requirements', '6.15Vendor andWarrantySpecifications', '6.16Surface Finish Requirements', '6.17StructuralIntegrity']]}
Metadata before chunk creation: {'filename': 'ProtonGlow_URS copy.pdf', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/ProtonGlow_URS copy.pdf', 'file_type': 'pdf', 'processed': '2025-02-16T11:12:53.362706'}


ERROR:root:Error extracting PDF tables: No module named 'camelot'


get_embeddings took 0.32 seconds to execute
content:{'text': 'Bejoy is a very good Boy', 'lists': []}
Metadata before chunk creation: {'filename': 'Bejoy is a very good Boy.docx', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/Bejoy is a very good Boy.docx', 'file_type': 'docx', 'processed': '2025-02-16T11:13:01.898132'}
get_embeddings took 0.15 seconds to execute
content:{'text': 'Bejoy is a smart boy', 'lists': []}
Metadata before chunk creation: {'filename': 'Bejoy is a smart boy.docx', 'file_path': '/content/drive/My Drive/lifesciences/training_documents/Bejoy is a smart boy.docx', 'file_type': 'docx', 'processed': '2025-02-16T11:13:02.110635'}
get_embeddings took 0.10 seconds to execute
INFO:     95.223.75.30:0 - "GET /train HTTP/1.1" 200 OK
get_embeddings took 0.08 seconds to execute
query took 0.09 seconds to execute
INFO:     95.223.75.30:0 - "POST /query HTTP/1.1" 200 OK
